# 等価変形とスケール変換

吉田勝俊（宇都宮大学）

## 参考情報

- [Pythonで運動方程式を解く(odeint) - Qiita](https://qiita.com/binaryneutronstar/items/ad5efa27fd626826846f)
- [[Python] Numpyの参照、抽出、結合 - Qiita](https://qiita.com/supersaiakujin/items/d63c73bb7b5aac43898a)

In [ ]:
import numpy as np                              #数値計算ライブラリ
from scipy.integrate import odeint              #常微分方程式ライブラリ
import matplotlib.pyplot as plt                 #描画ライブラリ
#Colab用の設定（グラフィックのインライン表示）
%matplotlib inline

## 演習4.2 (時間スケールの変換)

In [ ]:
def plot_function(func, ts, xlabel='t', key=''):
    '''
    時間関数のグラフを描画する関数
    '''
    vals = np.array([func(t) for t in ts])
    plt.plot(ts, vals, label=key)
    plt.xlabel(xlabel, fontsize=16)
    plt.ylabel('Value', fontsize=16)
    plt.legend()

### 時間関数のサンプル $x(t):=\sin(t)$

In [ ]:
def x(t):
    '''
    時間関数のサンプル
    '''
    return np.sin(t)

In [ ]:
ts = np.linspace(0,6,100) #時間軸

plot_function(x, ts, key='$x(t)$')

### 時間スケール変換

In [ ]:
def timescale_trans(func, alpha):
    '''
    時間関数 func(t) を時間スケール変換した関数を作る
    alpha は時間スケール
    '''
    def y(tau):
        '''
        時間スケール alpha で変換後の関数
        '''
        return x(alpha * tau)

    return y   

In [ ]:
alpha = 1.5 #時間スケールの一例
tau = (1/alpha) * ts #スケール変換後の時間軸

y = timescale_trans(x, alpha) #時間関数の時間スケール変換

plot_function(x, ts, key='$x(t)$') #元の関数形
plot_function(y, tau, key='$y(tau)$', xlabel='$t, tau$') #時間スケール変換後の関数形

* 時間軸方向のサイズが，`1/alpha = 1/1.5` 倍された．

## 演習4.3 (原形，正規形，標準形の数値解)

### 各運動方程式の定義

In [ ]:
def EOM_original(x, t, m, c, k):
    '''
    原形の運動方程式
    '''
    return np.array([
        x[1],
        -(c/m)*x[1] - (k/m)*x[0]
    ])

def EOM_canonical(x, t, zeta):
    '''
    正規形の運動方程式
    '''
    return np.array([
        x[1],
        -2*zeta*x[1] - x[0]
    ])

def EOM_standard(x, t, zeta, omega_n):
    '''
    標準形の運動方程式
    '''
    return np.array([
        x[1],
        -2*zeta*omega_n*x[1] - (omega_n**2)*x[0]
    ])

### 物理パラメータと無次元パラメータの設定

In [ ]:
m, c, k = 1.5, 1, 6 #物理パラメータの一例

zeta = c/(2*np.sqrt(m*k)) #対応する減衰比
omega_n = np.sqrt(k/m)    #対応する固有角振動数

print(omega_n)

### 時間軸

* $t=\alpha\tau$ $\implies$ $\tau = t/\alpha = \omega_n t$

In [ ]:
ts =  np.linspace(0,6,50) #物理時間軸
tau = omega_n * ts        #無次元時間軸

### 初期条件の設定

* この例では，$y(0)$, $\dot y(0)$ のほうを，基準として与える．
* $x$の初期変位は，時間スケール変換式(4.12)より，変位はそのまま$x(0)=y(0)$．
* $x$の初期速度は，$\dot x(0) = \frac{1}{\alpha}\dot y(0) = \omega_n \dot y(0)$．

In [ ]:
y0 = [1, 0]                  #y(tau)の初期変位，初期速度
x0 = [y0[0], omega_n*y0[1]]  #x(t)の初期変位，初期速度

### 数値解を求める

In [ ]:
x_original  = odeint(EOM_original,  x0, ts,  args=(m, c, k))       #原形
y_canonical = odeint(EOM_canonical, y0, tau, args=(zeta,))        #正規形
x_standard  = odeint(EOM_standard,  x0, ts,  args=(zeta, omega_n)) #標準形

### 変位の比較

In [ ]:
plt.figure(figsize=(10,3))
plt.plot(ts,   x_original[:,0], '-b', label=r'$x(t)$: original')
plt.plot(tau, y_canonical[:,0], '-k', label=r'$y(\tau)$: canonical')
plt.plot(ts,   x_standard[:,0], 'xr', label=r'$x(t)$: standard', ms=6)
plt.xlabel(r'$t,\tau$', fontsize=18)
plt.ylabel(r'$x(t),y(\tau)$', fontsize=18)
plt.grid()
plt.legend()

* 変位については，振幅方向は同サイズのまま，時間方向のサイズが $1/\omega_n=1/2$ 倍されている．

### 速度の比較

In [ ]:
plt.figure(figsize=(10,3))
plt.plot(ts,   x_original[:,1], '-b', label=r'$\dot x(t)$: original')
plt.plot(tau, y_canonical[:,1], '-k', label=r'$\dot y(\tau)$: canonical')
plt.plot(ts,   x_standard[:,1], 'xr', label=r'$\dot x(t)$: standard', ms=6)
plt.xlabel(r'$t,\tau$', fontsize=18)
plt.ylabel(r'$\dot x(t),\dot y(\tau)$', fontsize=18)
plt.grid()
plt.legend()

* 速度については，振幅方向のサイズが $\omega_n = 2$ 倍されている．（時間方向のサイズ変化は変位と同じ）